### OUTLINE

This notebook takes the force constants generated by Onetep and creates a FORCE_CONSTANTS file that can be ready by Phonopy. It then uses the Phonopy API to calculate phonon frequencies, eigenvectors , thermal properties etc...

It was also written to test that Phonopy and Onetep calculate the same phonon frequencies - a sanity check that both programmes are working correctly. A simple phonon calculation of ethene molecule is used.

At the moment these do not agree! Need to investigate : perhaps Onetep uses different units for forces?

#### TODO
- re-write f90 raw --> txt programme in Python
- parse unit cell data from onetep .dat file
- convert all this to a Python script

In [82]:
import numpy as np

from phonopy import Phonopy
from phonopy.structure.atoms import PhonopyAtoms
from phonopy.file_IO import parse_FORCE_CONSTANTS


In [ ]:
# should read this in from onetep .dat
num_atoms = 6

In [113]:
## read in the output data from onetep
## Note: A custom f90 programme has already been used to convert raw onetep output --> txt


# read all the data files in here so that we do not repeatedly read in as iterating through
fc_data_merged = []

# file each for atom_1 displaced in x, atom_1 displaced in y etc...
for i in range(3*num_atoms):
    with open("./data/ethene/force_constant_{}.dat".format(i+1),'r') as filein:
        fc_data_merged.append(np.array(filein.readlines()))
        


In [79]:
# create a four dimensional force constant array of size (num_atoms, num_atoms, 3, 3)  from the onetep data
# this could get slow - should refactor to eliminate for loops
# a sanity check is that the 2d arrays should symmetric.

fc_array = np.empty((num_atoms,num_atoms,3,3))

for i in range(num_atoms):
    for j in range(num_atoms):
        for k in range(3):
            data_file = fc_data_merged[3*i+k]
            fc_array[i,j,k] = np.array([float(data_file[j]),float(data_file[num_atoms+j]),float(data_file[2*num_atoms+j])])
            
       

In [ ]:
# Write the array to FORCE_CONSTANT file
i=1
j=1
with open('FORCE_CONSTANTS', 'w') as fileout:
    fileout.write('{0}   {1}\n'.format(num_atoms,num_atoms))

    # Iterating through a n-dimensional array produces slices of (n-1) dimensions
    # Print the resulting 2D arrays to file in FORCE_CONSTANT file format (https://atztogo.github.io/phonopy/input-files.html#force-constants-and-force-constants-hdf5)
    for three_dim_array in fc_array:
        for two_dim_array in three_dim_array:
            fileout.write('{0}   {1}\n'.format(i,j))
            np.savetxt(fileout,two_dim_array)
            if j < num_atoms:
                j +=1
            else:
                i+=1
                j=1

In [114]:
# Create a Phonopy object with unit cell basics. This could be read directly from onetep .dat file.

a = 40.0
unitcell = PhonopyAtoms(symbols=['C','H','H','C','H','H'],
                        cell=(np.eye(3) * a),
                        positions=[[21.243618,19.966152,20.054748],
                                   [22.327758,21.379927,21.104192],
                                   [22.335789,18.490615,19.103632],
                                   [18.756384,20.033161,19.945376],
                                   [17.664406,21.508773,20.896216], 
                                   [17.672044,18.619372,18.895836]])

phonon = Phonopy(unitcell,
                 [[1, 0, 0], [0, 1, 0], [0, 0, 1]])

In [142]:
# Parse the newly created FORCE_CONSTANTS file and calculate phonon properties using Phonopy API

phonon.set_force_constants(parse_FORCE_CONSTANTS())
mesh = [1,1,1]
phonon.set_mesh(mesh,run_immediately=True)
phonon.get_frequencies(0)


array([-0.04932665, -0.01964935, -0.01205844,  0.30372522,  0.48135503,
        1.0687199 ,  2.60674305,  3.03284525,  3.05573752,  3.32652768,
        3.69336319,  4.05755216,  4.31258497,  4.96844765,  9.31412146,
        9.33011153,  9.55315153,  9.63983278])

### Onetep calculated frequencies at gamma point:

--------------------------------(cm^-1)----------------------------------(THz)

    1:       -16.60902746844        -0.49792611698
    2:        -7.41759275439        -0.22237383643
    3:        -5.23600929839        -0.15697160977
    4:        99.82762136503         2.99275679853
    5:       158.32064999281         4.74633368135
    6:       351.43938017543        10.53588756208
    7:       857.13988318330        25.69640724294
    8:       997.24748334225        29.89672742655
    9:      1004.77428398580        30.12237523313
    10:      1093.81548046971        32.79176314885
    11:      1214.44398516073        36.40811474147
    12:      1334.19408061200        39.99813228757
    13:      1418.04468050917        42.51191003237
    14:      1633.70942245140        48.97737634145
    15:      3062.63405633412        91.81545917029
    16:      3067.89339339708        91.97313012885
    17:      3141.23226146954        94.17177408149
    18:      3169.73399144959        95.02623445028 